# **Lasso**

*Fonctionne avec Python 3.10.9 (Anaconda 23.3.1)*

## **Importation des bibliothèques**

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pour les traîtements sur les variables
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modèle de ML utilisé
from sklearn.linear_model import Lasso

# Pour gridsearch
from sklearn import model_selection

# Pour mesures des metrics des résultats
from sklearn import metrics

# Pour mise en forme des résultats
import colorama

## **Importation du dataset**

In [153]:
dataset_a_utiliser = "data_clean.xlsx"

data = pd.read_excel(dataset_a_utiliser)

## **Fonctions**

In [154]:
def split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification):

    ##############################
    # Split du dataset en X et y #
    ##############################

    global X
    global y

    X = data.loc[:,features_list] # On ajoute nos features dans le X
    y = data.loc[:,y_column] # On ajoute ce qu'on veut prédire dans le y

    ############################
    # Scaling et Encoding de X #
    ############################

    global feature_encoder

    numeric_transformer = StandardScaler()

    categorical_transformer = OneHotEncoder()

    feature_encoder = ColumnTransformer(
                transformers=[
                    ('cat', categorical_transformer, categorical_features),    
                    ('num', numeric_transformer, numeric_features)
                    ]
                )
    X = feature_encoder.fit_transform(X)


    ####################################
    # Split de X et y en train et test #
    ####################################

    global X_train
    global X_test
    global y_train
    global y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.2, 
                                                        random_state=0,
                                                        stratify=eval(stratification))

## **Features**

In [155]:
######################
# Choix des features #
######################

dataset_name = data

# Si rien mettre []
categorical_features = [
                        'BuildingType',
                        'PrimaryPropertyType',
                        'Neighborhood',
                        'ZipCode',
                        'YearBuilt'
                       ]

# Si rien mettre []
numeric_features = [
                    'NumberofBuildings',
                    'NumberofFloors',
                    'PropertyGFAParking',
                    'PropertyGFABuilding(s)',
                    'Latitude',
                    'Longitude',
                    'age_bat'
                   ]

# Toutes les features
features_list = categorical_features + numeric_features

## ***Lasso***

In [156]:
# Targets à estimer
targets = ['SiteEnergyUse_kBtu', 'TotalGHGEmissions']

# "y" pour stratifier y, sinon "None"
stratification = "None"

# Afficher les résultats détaillés (True/False)
details = False

for i in targets:

    y_column=i

    print(f"=========== [{i}] ===========")


    # Split & Scale du dataset
    split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification)


    # Entraînement simple
    modele = Lasso() # Paramètres par défaut
    modele.fit(X_train, y_train) # Étape d'entraînement
    print("--- Entraînement simple : ---")
    print(f"Scores du modèle pour prédire {i} :")
    y_pred = modele.predict(X_test)
    print(f"R² : {modele.score(X_train, y_train):.3f} (train) et {colorama.Style.BRIGHT}{colorama.Back.CYAN}{colorama.Fore.BLACK} {modele.score(X_test, y_test):.3f} {colorama.Style.RESET_ALL} (test)")
    print(f"MSE : {metrics.mean_squared_error(y_test, y_pred):.4}")
    print(f"MAE : {metrics.mean_absolute_error(y_test, y_pred):.4}")


    ###############################################
    # Paramétrage de GridSearchCV et entraînement #
    ###############################################

    print("--- GridSearch : ---")

    # Fixer les valeurs des hyperparamètres à tester
    param_grid = {'alpha':[0.5, 1, 2, 3, 3.5, 4, 5, 10, 100, 1000, 10000, 10000, 60000]} #, 'max_iter':[50, 100, 1000]

    # Déterminer le score qu'on veut optimiser
    score = 'r2'

    # Je le sors ici car je vais réutiliser cette valeur plus tard
    cv = 5

    grid = model_selection.GridSearchCV(
        Lasso(fit_intercept=False), # On indique le modèle à tester
        param_grid,     # hyperparamètres à tester
        cv=cv,           # nombre de folds de validation croisée
        scoring=score   # score à optimiser
    )

    # Optimiser ce modèle sur le jeu d'entraînement
    grid.fit(X_train, y_train)


    ###########################
    # Affichage des résultats #
    ###########################

    # Afficher le(s) hyperparamètre(s) optimaux
    print(f"Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement ({i}): {grid.best_params_}")

    # Afficher les performances correspondantes
    if details == True:
        print('Résultats pour chaque fold :')
        for j in range(cv):
            print(f"Fold n°{j+1} :", eval(f"grid.cv_results_['split{j}_test_score']"), f"(Pour les paramètres : {grid.cv_results_['params']})")

        print("\nRésultats de la validation croisée :")
        for mean, std, params in zip(
                grid.cv_results_['mean_test_score'], # score moyen
                grid.cv_results_['std_test_score'],  # écart-type du score
                grid.cv_results_['params']           # valeur de l'hyperparamètre
            ):

            print(f"{score} (moyen) : {mean:.04f} (+/-{std*2:.04f}) pour {params}")


    ###################################################################
    # Prédiction sur le jeu de test avec les hyperparamètres optimaux #
    ###################################################################

    # GridSearchCV a automatiquement ré-entraîné le meilleur modèle sur l’intégralité du jeu d’entraînement.
    y_pred = grid.predict(X_test)
    print(f"Score sur le jeu de test ({i}, avec paramètres optimaux) :")
    print(f"R² : {colorama.Style.BRIGHT}{colorama.Back.GREEN}{colorama.Fore.BLACK} {metrics.r2_score(y_test, y_pred):.3f} {colorama.Style.RESET_ALL}")
    print(f"MSE : {metrics.mean_squared_error(y_test, y_pred):.4}")
    print(f"MAE : {metrics.mean_absolute_error(y_test, y_pred):.4}")

    print('_'*80, "\n")

    

=========== [SiteEnergyUse_kBtu] ===========


C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.3099187873081116e+16, tolerance: 22349412502619.2
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.032316468408595e+16, tolerance: 24654374140952.77
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.2087557822099172e+16, tolerance: 23585916129249.625
  model = cd_fast.sparse_enet_coordinate_desce

--- Entraînement simple : ---
Scores du modèle pour prédire SiteEnergyUse_kBtu :
R² : 0.733 (train) et  0.564  (test)
MSE : 7.912e+13
MAE : 4.6e+06
--- GridSearch : ---


C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.127962689553115e+16, tolerance: 23842933464606.3
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.0162693317255148e+16, tolerance: 24654374140952.77
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\Data Science\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.206287493997107e+16, tolerance: 23585916129249.625
  model = cd_fast.sparse_enet_coordinate_descen

Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (SiteEnergyUse_kBtu): {'alpha': 60000}
Score sur le jeu de test (SiteEnergyUse_kBtu, avec paramètres optimaux) :
R² :  0.618 
MSE : 6.937e+13
MAE : 3.749e+06
________________________________________________________________________________ 

=========== [TotalGHGEmissions] ===========
--- Entraînement simple : ---
Scores du modèle pour prédire TotalGHGEmissions :
R² : 0.560 (train) et  0.385  (test)
MSE : 5.857e+04
MAE : 114.1
--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (TotalGHGEmissions): {'alpha': 2}
Score sur le jeu de test (TotalGHGEmissions, avec paramètres optimaux) :
R² :  0.390 
MSE : 5.815e+04
MAE : 110.2
________________________________________________________________________________ 

